In [2]:
library(tidyverse)
library(rvest)

covid_url <- "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv"

deaths_by_county <- read_csv(covid_url, show_col_types = FALSE) %>%
      mutate(CountyState = paste(Admin2, Province_State, sep = ', ')) %>%
      rename("Deaths" = "12/31/21") %>%
      select(CountyState, Deaths) 

gdp_table <- read_html("https://en.wikipedia.org/wiki/List_of_United_States_counties_by_per_capita_income") %>%
        html_element(xpath = "//table[3]") %>%
        html_table()

colnames(gdp_table) <- c("Rank", "County", "State",
        "PerCapitaIncome", "MedianHousehold", "MedianFamily", "Population", "NumHouseholds")

gdp_table <- gdp_table %>%
              filter(Rank != "—") %>%
              mutate(CountyState = paste(str_trim(County), State, sep = ', '),
                    PerCapitaIncome = parse_number(PerCapitaIncome),
                    MedianHousehold = parse_number(MedianHousehold),
                    MedianFamily = parse_number(MedianFamily),
                    Population = parse_number(Population),
                    NumHouseholds = parse_number(NumHouseholds),
                    Rank = parse_number(Rank)
              )

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   2.1.1     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘rvest’


The following object is masked from ‘package:readr’:

    guess_encoding




In [ ]:
merged <- left_join(deaths_by_county, gdp_table, by = c("CountyState" = "CountyState")) %>%
        mutate(PerCapitaDeaths = Deaths / Population)

head(merged)

In [ ]:
mg <- merged %>% filter(is.na(PerCapitaDeaths))


